In [15]:
from matplotlib import pyplot as plt
from matplotlib.image import imread
import numpy as np
import os
import cv2
from collections import defaultdict
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report,accuracy_score



In [16]:
class FaceRecognition:

    def __init__(self):
        
        self.TRAIN_IMG_FOLDER = './images/TestDetected/'
        self.train_imgs_names = os.listdir(self.TRAIN_IMG_FOLDER)
        self.label_dict = defaultdict(int)
        # amira 1/doha 2/maha 3/mariam 4/mayar 5
        for name in self.train_imgs_names:
            label = name.split('_')[0]
            if label not in self.label_dict:
                self.label_dict[label] = len(self.label_dict) + 1

        self.width = 128
        self.height = 128
        self.mean_face = np.zeros((1,self.height * self.width))
        self.Proj_Training_Matrix = None
        self.eigenvalues = None
        self.eigenvectors = None
        self.weights = None
        self.best_threshold = None


    def pca(self):

        Training_Matrix   = np.ndarray(shape=(len(self.train_imgs_names), self.height * self.width), dtype=np.float64)
        for i in range(len( self.train_imgs_names)):

            img = cv2.imread(self.TRAIN_IMG_FOLDER +  self.train_imgs_names[i], cv2.IMREAD_GRAYSCALE)
            Training_Matrix[i,:] = np.array(img, dtype='float64').flatten()


        for i in Training_Matrix:
             self.mean_face = np.add(self.mean_face,i)
        self.mean_face = np.divide(self.mean_face,float(Training_Matrix.shape[0])).flatten()

        Normalised_Training_Matrix = np.ndarray(shape=(Training_Matrix.shape[0], Training_Matrix.shape[1]))
        for i in range(len(self.train_imgs_names)):
            Normalised_Training_Matrix[i] = np.subtract(Training_Matrix[i],self.mean_face)
        cov_matrix=np.cov(Normalised_Training_Matrix)
        cov_matrix = np.divide(cov_matrix,float(len(self.train_imgs_names)))  
        self.eigenvalues, self.eigenvectors, = np.linalg.eig(cov_matrix)

        sorted_ind = sorted(range(self.eigenvalues.shape[0]), key=lambda k: self.eigenvalues[k], reverse=True) 
        self.eigenvalues = self.eigenvalues[sorted_ind]
        self.eigenvectors = self.eigenvectors[sorted_ind]  

        # var_comp_sum = np.cumsum(self.eigenvalues)/sum(self.eigenvalues)
        # for num_comp in range(1, len(self.eigenvalues) + 1):
        #     if var_comp_sum[num_comp - 1] >= 0.9:
        #         break

        # reduced_eigvectors = np.array(self.eigenvectors[:num_comp]).transpose()
        # self.Proj_Training_Matrix = np.dot(Training_Matrix.transpose(),reduced_eigvectors)

        self.Proj_Training_Matrix = np.dot(Training_Matrix.transpose(),self.eigenvectors)
        self.Proj_Training_Matrix =  self.Proj_Training_Matrix.transpose()

        self.weights = np.array([np.dot( self.Proj_Training_Matrix,i) for i in Normalised_Training_Matrix])


    def threshold_range(self):

        euclidean_values = []
        for img in self.test_imgs_names:

            unknown_face = cv2.imread(self.TEST_IMG_FOLDER+img,0)        
            unknown_face_vector = np.array(unknown_face, dtype='float64').flatten()
            normalised_uface_vector = np.subtract(unknown_face_vector,self.mean_face)

            unknown_weights = np.dot(self.Proj_Training_Matrix, normalised_uface_vector)
            diff  = self.weights - unknown_weights
            Euclidean_dist = np.linalg.norm(diff, axis=1)
            index = np.argmin(Euclidean_dist)
            euclidean_values.append( Euclidean_dist[index])
        return( min(euclidean_values) , max(euclidean_values) )
    


    
    def train(self):
        # Load and preprocess the training images
        X_train = []
        y_train = []
        for name in self.train_imgs_names:
            img = cv2.imread(self.TRAIN_IMG_FOLDER + name, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (self.width, self.height))
            X_train.append(img.flatten())
            label = self.label_dict[name.split('_')[0]]
            y_train.append(label)
        print(y_train)
        # Perform PCA on the training images
        self.pca()

        # Project the training images onto the PCA subspace
        X_train_proj = np.dot(self.Proj_Training_Matrix, np.array(X_train).T).T

        param_grid = {'kernel' : ('poly', 'rbf'), 'C':[0.01, 0.1, 0.5, 1, 10,50,100,1000],'gamma':[0.1,0.01,0.001,0.0001,0.00001,0.000001]}
        gs = GridSearchCV(estimator=SVC(kernel='rbf'),
        param_grid=param_grid,
        refit=True, # choose best model and refit to entire data
        n_jobs=None)
        gs.fit(X_train_proj,y_train)

        best_params = gs.best_params_
        print('Best Params:', best_params)
        bestModel = SVC(kernel=best_params['kernel'],C=best_params['C'])
        bestModel.fit(X_train_proj,y_train)
        print('Accuracy of SVC on training set: {:.2f}'.format(bestModel.score(Xtrain_pca,y_train)*100))
        print('Accuracy of SVC on test set: {:.2f}'.format(bestModel.score(Xtest_pca,y_test)*100))


        print(accuracy_score(y_test,y_pred))

        # Train an SVM classifier on the projected training images
        self.classifier = SVC(kernel='rbf', C=1, gamma='auto')
        self.classifier.fit(X_train_proj, y_train)

    def recognize(self, img_path):
        # Load and preprocess the test image
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (self.width, self.height))
        img_vec = np.array(img, dtype='float64').flatten()
        img_vec_norm = np.subtract(img_vec, self.mean_face)

        # Project the test image onto the PCA subspace
        img_proj = np.dot(self.Proj_Training_Matrix, img_vec_norm)
        # Predict the label of the test image using the k-NN classifier
        label = self.classifier.predict([img_proj])[0]
        # Return the name of the predicted person
        for name, label_int in self.label_dict.items():
            if label_int == label:
                return name
        return 'Unknown'
    

            


In [17]:
f = FaceRecognition()
f.train()
f.label_dict

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6]


defaultdict(int,
            {'amira': 1,
             'doha': 2,
             'maha': 3,
             'mariam': 4,
             'mayar': 5,
             'unknown': 6})

In [18]:
TEST_IMG_FOLDER = 'E:/THIRD YEAR/Second term/cv/tasks/task5/images/TestDetected/'
test_image_names = os.listdir(TEST_IMG_FOLDER)
for i in range(len(test_image_names)):
   print(test_image_names[i] ,f.recognize(TEST_IMG_FOLDER + test_image_names[i]))

amira_1.jpeg doha
amira_11.jpeg doha
amira_12.jpeg doha
amira_2.jpeg doha
amira_3.jpeg doha
amira_4.jpeg doha
amira_5.jpeg doha
amira_6.jpeg doha
amira_7.jpeg doha
amira_8.jpeg doha
amira_9.jpeg doha
doha_1.jpeg doha
doha_10.jpeg doha
doha_11.jpeg doha
doha_12.jpeg doha
doha_13.jpeg doha
doha_14.jpeg doha
doha_2.jpeg doha
doha_3.jpeg doha
doha_4.jpeg doha
doha_5.jpeg doha
doha_6.jpeg doha
doha_7.jpeg doha
doha_8.jpeg doha
doha_9.jpeg doha
maha_1.jpeg doha
maha_10.jpeg doha
maha_11.jpeg doha
maha_12.jpeg doha
maha_2.jpeg doha
maha_3.jpeg doha
maha_4.jpeg doha
maha_5.jpeg doha
maha_6.jpeg doha
maha_7.jpeg doha
maha_8.jpeg doha
maha_9.jpeg doha
mariam_1.jpg doha
mariam_10.jpg doha
mariam_11.jpg doha
mariam_2.jpg doha
mariam_3.jpg doha
mariam_4.jpg doha
mariam_5.jpg doha
mariam_6.jpg doha
mariam_7.jpg doha
mariam_8.jpg doha
mariam_9.jpg doha
mayar_10.jpeg doha
mayar_11.jpeg doha
mayar_12.jpeg doha
mayar_2.jpeg doha
mayar_3.jpeg doha
mayar_4.jpeg doha
mayar_5.jpeg doha
mayar_6.jpeg doha
may